In [1]:
import sys; sys.path.append("../../../automl/")

In [2]:
from pathlib import Path
import yaml
import joblib

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from src.automl.model.lama import TabularLamaNN
from src.automl.loggers import configure_root_logger
from src.automl.constants import create_ml_data_dir
from src.automl.model.metrics import RocAuc

'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'gensim' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.
'nlp' extra dependecy package 'transformers' isn't installed. Look at README.md in repo 'LightAutoML' for installation instructions.


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/lightautoml/ml_algo/dl_model.py:42: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/opt/conda/lib/python3.10/site-packages/lightautoml/text/embed.py:22: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")
/opt/conda/lib/python3.10/site-packages/lightautoml/text/dl_transformers.py:25: UserWarning: 'transformers' - package isn't installed
  warnings.warn("'transformers' - package isn't installed")


In [3]:
create_ml_data_dir()
configure_root_logger()

## Constants

In [4]:
RANDOM_SEED = 77
DATA_PATH = Path("../../../data/")
CONFIG_PATH = Path("../../../configs/config.yaml")
N_JOBS = 5

In [5]:
with CONFIG_PATH.open() as f:
    cfg = yaml.load(f, Loader=yaml.SafeLoader)

In [6]:
torch.cuda.is_available()

True

## Data

In [7]:
df_train = pd.read_parquet(DATA_PATH / "train_preproc_2.parquet")
#df_train, df_test = df_train.sort_values(by="id").iloc[:300_000], df_train.sort_values(by="id").iloc[300_000:]

In [8]:
df_train["target"].value_counts(normalize=True)

0    0.940982
1    0.059018
Name: target, dtype: float64

In [9]:
# undersample the 0 class
#df_train = pd.concat([df_train.loc[df_train.target == 1], df_train.loc[df_train.target == 0].sample(200_000, random_state=RANDOM_SEED)], ignore_index=True)


In [10]:
cat_columns = df_train.drop(columns=["target", "id"]).select_dtypes(int).columns.values.tolist()

In [11]:
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]
#X_test, y_test = df_test[cfg["selected_features"] + cat_columns], df_test["target"]

In [12]:
display(y_train.value_counts(normalize=True))
#display(y_test.value_counts(normalize=True))

0    0.940982
1    0.059018
Name: target, dtype: float64

In [13]:
#categorical_features = ohe_cols# + oe_cols

## Model

In [14]:
metric = RocAuc()

In [15]:
model = TabularLamaNN(n_jobs=16, task="classification", nn_name="autoint")
model.tune(X_train, y_train, metric, timeout=60 * 60 * 2, categorical_features=cat_columns)
model.verbose = 4
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 12:08:58,759] - [   START    ] - Fitting TabularLamaNN_autoint
[12:08:58] Stdout logging level is DEBUG.
[12:08:58] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[12:08:58] Task: binary

[12:08:58] Start automl preset with listed constraints:
[12:08:58] - time: 14400.00 seconds
[12:08:58] - CPU: 16 cores
[12:08:58] - memory: 16 GB

[12:08:58] Train data shape: (413194, 63)

[12:09:14] Feats was rejected during automatic roles guess: []
[12:09:14] Layer 1 train process start. Time left 14384.51 secs
[12:09:23] number of text features: 0 
[12:09:23] number of categorical features: 4 
[12:09:23] number of continuous features: 58 
[12:09:23] Start fitting Lvl_0_Pipe_0_Mod_0_TorchNN_autoint_0 ...
[12:09:23] Training params: {'num_workers': 0, 'pin_memory': False, 'max_length': 256, 'is_snap': False, 'input_bn': False, 'max_emb_size': 256, 'bert_name': None, 'pooling': 'cls', 'device': device(type='cuda', index=0), 'use_cont': 

[12:40:28] Epoch: 23, train loss: 0.183428555727005, val loss: 0.19311760365962982, val metric: 0.7957049808879094
[12:41:06] Epoch: 24, train loss: 0.18255187571048737, val loss: 0.19092701375484467, val metric: 0.7976241610330523
[12:41:43] Epoch: 25, train loss: 0.1813998967409134, val loss: 0.1934359222650528, val metric: 0.7935573166826069
[12:42:21] Epoch: 26, train loss: 0.17998187243938446, val loss: 0.19299423694610596, val metric: 0.7897826605480652
[12:42:59] Epoch: 27, train loss: 0.1777971237897873, val loss: 0.19352443516254425, val metric: 0.791000470054638
[12:43:36] Epoch: 28, train loss: 0.17629578709602356, val loss: 0.19533847272396088, val metric: 0.7896064223597945
[12:44:14] Epoch: 29, train loss: 0.17584431171417236, val loss: 0.19515475630760193, val metric: 0.7886150705202986
[12:44:52] Epoch: 30, train loss: 0.17505429685115814, val loss: 0.19657976925373077, val metric: 0.7848700337396953
[12:45:29] Epoch: 31, train loss: 0.1741061806678772, val loss: 0.1970

[13:23:32] Epoch: 30, train loss: 0.17395038902759552, val loss: 0.19658687710762024, val metric: 0.7908451313844037
[13:24:10] Epoch: 31, train loss: 0.17376229166984558, val loss: 0.20260027050971985, val metric: 0.7837468566427026
[13:24:16] Early stopping: val loss: 0.18829074501991272, val metric: 0.8059301342489384
[13:24:16] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_TorchNN_autoint_0 =====
[13:24:54] Epoch: 0, train loss: 2.0756165981292725, val loss: 0.29272064566612244, val metric: 0.6760281684864917
[13:25:32] Epoch: 1, train loss: 1.0410487651824951, val loss: 0.2044340819120407, val metric: 0.761761622140692
[13:26:09] Epoch: 2, train loss: 0.1988583207130432, val loss: 0.19150947034358978, val metric: 0.7986280810163797
[13:26:47] Epoch: 3, train loss: 0.410273015499115, val loss: 0.19233688712120056, val metric: 0.7967691374397543
[13:27:25] Epoch: 4, train loss: 0.4932568371295929, val loss: 0.18968094885349274, val metric: 0.8010486340673248
[13:28:02] Epoc

In [16]:
print(metric(y_train, oof))

0.8053888481876601


In [17]:
MODEL_NAME = "lamann_autoint_8053_full_dataset"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [18]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

In [24]:
imp = pd.DataFrame().assign(names=model.models[0].feature_names_, imp=model.models[0].feature_importances_)

In [41]:
imp.sort_values(by="imp", ascending=False).reset_index(drop=True).query("names == 'feature_185'")

,names,imp
61,feature_185,0.0


In [29]:
cat_columns

['feature_7',
 'feature_31',
 'feature_60',
 'feature_61',
 'feature_71',
 'feature_109',
 'feature_122',
 'feature_156',
 'feature_163',
 'feature_167',
 'feature_179',
 'feature_185']

## With Time series cross val

In [13]:
metric = RocAuc()

In [14]:
df_train = df_train.sort_values(by="id").reset_index(drop=True)
X_train, y_train = df_train[cfg["selected_features"] + cat_columns], df_train["target"]

In [15]:
model = LightGBMClassification(n_jobs=16, time_series=True)
model.tune(X_train, y_train, metric, timeout=60 * 60, categorical_features=cat_columns)
oof = model.fit(X_train, y_train, categorical_features=cat_columns)

print(metric(y_train, oof))

[2024-11-07 08:27:33,388] - [   START    ] - Tuning LightGBMClassification
[2024-11-07 08:27:40,118] - [   OPTUNA   ] - Trial 0. New best score 0.7903405446081995 with parameters {'max_depth': 6, 'num_leaves': 488, 'min_data_in_leaf': 188, 'bagging_fraction': 0.7993292420985183, 'bagging_freq': 0, 'feature_fraction': 0.49359671220172163, 'lambda_l1': 0.5808361216819946, 'lambda_l2': 8.661761457749352, 'min_gain_to_split': 12.022300234864176, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:03,043] - [   OPTUNA   ] - Trial 2. New best score 0.7913515589848906 with parameters {'max_depth': 5, 'num_leaves': 194, 'min_data_in_leaf': 117, 'bagging_fraction': 0.8925879806965068, 'bagging_freq': 0, 'feature_fraction': 0.708540663048167, 'lambda_l1': 5.924145688620425, 'lambda_l2': 0.46450412719997725, 'min_gain_to_split': 12.150897038028766, 'is_unbalance': True, 'num_iterations': 2}
[2024-11-07 08:28:12,225] - [   OPTUNA   ] - Trial 3. New best score 0.8048686053278628 with param

In [27]:
none_oofs_idx = oof[np.any(np.isnan(oof), axis=1)].shape[0]

In [32]:
metric(y_train[none_oofs_idx:], oof[none_oofs_idx:])

0.8095227594190041

In [34]:
MODEL_NAME = "lgb_8095_full_dataset_time_series"
MODEL_DIR = Path(f"../../../data/models/{MODEL_NAME}")
MODEL_DIR.mkdir(exist_ok=True)

In [35]:
res = pd.DataFrame()
res[MODEL_NAME] = oof[none_oofs_idx:, 1]
res.to_csv(MODEL_DIR / "oof.csv", index=False)
joblib.dump(model, MODEL_DIR / f"{MODEL_NAME}.joblib")

with (MODEL_DIR / "params.yaml").open("w") as f:
    yaml.dump(model.params, f)

with (MODEL_DIR / "score.txt").open("w") as f:
    print("OOF:", metric(y_train, oof), file=f)
    
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv(MODEL_DIR / f'{MODEL_NAME}.csv', index=False)

## TEST 
**81.22112399468679**

## Inference

In [27]:
test = pd.read_parquet(DATA_PATH / "test_preproc_2.parquet")
test["target"] = model.predict(test[cfg["selected_features"] + cat_columns])[:, 1]
test[['id', 'target']].to_csv('lgb_813.csv', index=False)

In [25]:
pred_1 = pd.read_csv("lama_utilized.csv")
pred_2 = pd.read_csv("lgmb_oe_ohe_cols_0805.csv")
pred_3 = pd.read_csv("catboost_ts.csv")

In [26]:
pred_1["target"] = 0.6 * pred_1["target"] + 0.2 * pred_2["target"] + 0.2 * pred_3["target"]

In [29]:
pred_1.to_csv("blend.csv", index=False)

In [166]:
MODEL_DIR.open?

Signature:
MODEL_DIR.open(
    mode='r',
    buffering=-1,
    encoding=None,
    errors=None,
    newline=None,
)
Docstring:
Open the file pointed by this path and return a file object, as
the built-in open() function does.
File:      /usr/lib/python3.10/pathlib.py
Type:      method